$$
\newcommand{theorem}{\textbf{Theorem: }}
\newcommand{corollary}{\textbf{Corollary: }}
$$

In [1]:
import sys

sys.path.append("..")

import numpy as np
from module.utility import print_table, make_vars
from common.utility import show_implementation

# Sensitivity analysis

Suppose that we have computed the optimal solution to some LP.
Now, there is some changes made to the parameter of the LP.
Rather than re-computing the whole problem again, is it possible for us to use our previous result to derive the new optimal solution?
**Sensitivity analysis** aims to answer this problem.

We will analyze the following list of parameters that can change in an LP:
* Change in $\mathbf b$
* Change in $\mathbf c$
* Change in a non basic column of $\mathbf A$
* Adding a new variable
* Adding a new constraint

The two core conditions that we are interested in are:

Feasibility: $\mathbf B ^{-1} \mathbf b \geq \mathbf 0$

Optimality: $\bar {\mathbf c} = \mathbf c^T - \mathbf c^T_B \mathbf B^{-1} \mathbf A \geq 0$

In fact, to solve for the new LP, all we need to do is to perform simplex iterations if it is not optimal; perform dual simplex iterations if it is not feasible.

## Change in $\mathbf b$

Suppose that we perturbed the problem such that our constraints are
$$
\mathbf {Ax} = \mathbf b + \Delta
$$

If $\mathbf{B}^{-1} (\mathbf b + \Delta) \geq \mathbf 0$, *ie* the problem is still feasible, then notice that $\bar{ \mathbf c}$ remains unchanged as it is not a function of $\mathbf b$.
Therefore, the basis still leads to an optimal solution.
All that is left is to compute the optimal cost:
$$
\mathbf c^T_B \mathbf B ^{-1}(\mathbf b + \Delta) = 
\mathbf c^T_B \mathbf B ^{-1}\mathbf b + \mathbf c^T_B \mathbf B ^{-1}\Delta = 
\mathbf c^T_B \mathbf B ^{-1}\mathbf b + \mathbf p ^T \Delta
$$

In other words, the change in $\mathbf b$ led to a change of $\mathbf p ^T \Delta$ to the objective.

However, if $\Delta$ is such that feasibility becomes violated, then we need to perform [dual simplex iterations](./duality.ipynb#Dual-Simplex-method) to regain feasibility and find the new optimal solution.

### Marginal cost

Suppose that $\Delta = e_i$, *ie* the [elementary vector](TODO).
Then 
$$
\mathbf p ^T \Delta = p_i
$$
We call the term $p_i$ the **marginal cost/shadow cost** of $b_i$

They represent the amount of objective value we can increase for every unit we increase $b_i$, hence they can be used to rank which resource should be expanded to better improve cost.

### Example

We will once again revisit our [application problem](introduction.ipynb#Application).

| Resource/Activity | Playing games | Studying | Eating | Napping | Amount available |
| --- | --- | --- | --- | --- | --- |
| Time | 2 | 1 | 0.5 | 1 | 8 |
| Energy | 2 | 1 | 0.5 | -3 | 4 |
| Hunger | 1 | 1 | -2 | 0.5 | 6 |
| Stress | -3 | 2 | 0 | -1 | 3 |
| Happiness | 2 | 1 | 0.5 | 0.5 | |

We obtain the below tableau at the end of our simplex method


|Basis| x1 | x2 | x3 | x4 | s1 | s2 | s3  | s4   | Solution | 
| --- |--- |--- |--- |--- |--- |--- | --- | --- | --- |
| $$\mathbf c$$ | 0| 0 | 0 | 0 | 0.875 | 0.125| 0 | 0 | 7.5 |
| x4 | 0 | 0 | 0 | 1 | 0.25 | -0.25 | 0 | 0 | 1|
| x1 | 1 | 0.5 | 0.25 | 0 | 0.375 | 0.125 | 0 | 0| 3.5 |
| s3 | 0 | 0.5 | -2.25 | 0 | -0.5 | 0 | 1 | 0 | 2 |
| s4 | 0 | 3.5 | 0.75 | 0 | 1.375 | 0.125 | 0 | 1 | 14.5 |

In [2]:
orig_costs = np.array([-2, -1, -0.5, -0.5, 0, 0, 0, 0, 0])
orig_tableau = np.array(
    [
        [2, 1, 0.5, 1, 1, 0, 0, 0, 8],
        [2, 1, 0.5, -3, 0, 1, 0, 0, 4],
        [1, 1, -2, 0.5, 0, 0, 1, 0, 6],
        [-3, 2, 0, -1, 0, 0, 0, 1, 3],
    ]
)

sol_c_bar = np.array([0, 0, 0, 0, 0.875, 0.125, 0, 0, 7.5])
sol_tableau = np.array(
    [
        [0.0, 0.0, 0.0, 1.0, 0.25, -0.25, 0.0, 0.0, 1],
        [1.0, 0.5, 0.25, 0.0, 0.375, 0.125, 0.0, 0.0, 3.5],
        [0.0, 0.5, -2.25, 0.0, -0.5, 0.0, 1.0, 0.0, 2],
        [0.0, 3.5, 0.75, 0.0, 1.375, 0.125, 0.0, 1.0, 14.5],
    ]
)

In [3]:
variables = make_vars("x", 4) + make_vars("s", 4)
print_table(sol_c_bar, sol_tableau, variables)

$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0.875$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$ 7.5$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 1$$
$$x_1$$,$$ 1$$,$$ 0.5$$,$$ 0.25$$,$$ 0$$,$$ 0.375$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$ 3.5$$
$$s_3$$,$$ 0$$,$$ 0.5$$,$$ -2.25$$,$$ 0$$,$$ -0.5$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 2$$
$$s_4$$,$$ 0$$,$$ 3.5$$,$$ 0.75$$,$$ 0$$,$$ 1.375$$,$$ 0.125$$,$$ 0$$,$$ 1$$,$$ 14.5$$


Computing the shadow costs we get
$$
\mathbf p^T = \mathbf c^T_B \mathbf B^{-1} =
\begin{pmatrix}
-0.5 & -2 & 0 &0
\end{pmatrix}
\begin{pmatrix}
0.25 & -0.25 & 0 & 0 \\
0.375 & 0.125 & 0 & 0 \\
-0.5 & 0 & 1 & 0 \\
1.375 & 0.125 & 0 & 1
\end{pmatrix}
=
\begin{pmatrix}
-0.875 & -0.125 & 0 & 0
\end{pmatrix}
$$

This means that we would obtain higher happiness if we increased our time or energy.
On the flip-side, we have that increasing our hunger or our stress capacity doesn't increase our happiness, because these resources are not fully utilized in the optimal solution, and thus are not the limiting factor.


---

Suppose that we increased our units of time from $8$ to $13$.

In [4]:
from module.simplex import reduced_cost, simplex

new_tableau = orig_tableau.copy()
new_tableau[0, -1] = 13
c_bar = reduced_cost(orig_costs, new_tableau)
print("New initial tableua")
print_table(c_bar, new_tableau, variables, tableau_bold=(0, 8, 1, 1))

print("Final tableau")
print_table(*simplex(c_bar, new_tableau), variables)

New initial tableua


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ -2$$,$$ -1$$,$$ -0.5$$,$$ -0.5$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$
$$s_1$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ 1$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$\bf 13$$
$$s_2$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ -3$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 4$$
$$s_3$$,$$ 1$$,$$ 1$$,$$ -2$$,$$ 0.5$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 6$$
$$s_4$$,$$ -3$$,$$ 2$$,$$ 0$$,$$ -1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 3$$


Final tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0.875$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$ 11.875$$
$$x_3$$,$$ 0$$,$$ -0.2222$$,$$ 1$$,$$ 0$$,$$ 0.2222$$,$$ 0$$,$$ -0.4444$$,$$ 0$$,$$ 0.2222$$
$$x_1$$,$$ 1$$,$$ 0.5556$$,$$ 0$$,$$ 0$$,$$ 0.3194$$,$$ 0.125$$,$$ 0.1111$$,$$ 0$$,$$ 5.3194$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 2.25$$
$$s_4$$,$$ 0$$,$$ 3.6667$$,$$ 0$$,$$ 0$$,$$ 1.2083$$,$$ 0.125$$,$$ 0.3333$$,$$ 1$$,$$ 21.2083$$


Now, it is better to game and eat instead.

The question becomes, can we arrive at this new solution from the old one?

Firstly, we compute the $\mathbf B^{-1} \mathbf b$
$$
\mathbf B^{-1} \mathbf b  =
\begin{pmatrix}
0.25 & -0.25 & 0 & 0 \\
0.375 & 0.125 & 0 & 0 \\
-0.5 & 0 & 1 & 0 \\
1.375 & 0.125 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
13 \\ 4 \\ 6 \\ 3
\end{pmatrix}
=
\begin{pmatrix}
2.25 \\ 5.375 \\ -0.5 \\ 21.375
\end{pmatrix}
$$

Indeed, the $-0.5$ indicated that our basis is no longer optimal.
Our new tableau is as follows:


|Basis| x1 | x2 | x3 | x4 | s1 | s2 | s3  | s4   | Solution | 
| --- |--- |--- |--- |--- |--- |--- | --- | --- | --- |
| $$\mathbf c$$ | 0| 0 | 0 | 0 | 0.875 | 0.125| 0 | 0 |  |
| x4 | 0 | 0 | 0 | 1 | 0.25 | -0.25 | 0 | 0 | 2.25 |
| x1 | 1 | 0.5 | 0.25 | 0 | 0.375 | 0.125 | 0 | 0| 5.375 |
| s3 | 0 | 0.5 | -2.25 | 0 | -0.5 | 0 | 1 | 0 | -0.5 |
| s4 | 0 | 3.5 | 0.75 | 0 | 1.375 | 0.125 | 0 | 1 | 21.375 |

Then we just perform dual simplex iterations on this tableau.

In [5]:
from module.simplex import dual_simplex

new_tableau = sol_tableau.copy()
new_tableau[:, -1] = [2.25, 5.375, -0.5, 21.375]
c_bar = reduced_cost(orig_costs, new_tableau)

print("Perturbed tableau")
print_table(c_bar, new_tableau, variables, c_bold=[8], tableau_bold=(0, 8, 4, 1))

print("Final tableau")
print_table(*dual_simplex(c_bar, new_tableau), variables)

Perturbed tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0.875$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$\bf 11.875$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$\bf 2.25$$
$$x_1$$,$$ 1$$,$$ 0.5$$,$$ 0.25$$,$$ 0$$,$$ 0.375$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$\bf 5.375$$
$$s_3$$,$$ 0$$,$$ 0.5$$,$$ -2.25$$,$$ 0$$,$$ -0.5$$,$$ 0$$,$$ 1$$,$$ 0$$,$$\bf -0.5$$
$$s_4$$,$$ 0$$,$$ 3.5$$,$$ 0.75$$,$$ 0$$,$$ 1.375$$,$$ 0.125$$,$$ 0$$,$$ 1$$,$$\bf 21.375$$


Final tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0.875$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$ 11.875$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 2.25$$
$$x_1$$,$$ 1$$,$$ 0.5556$$,$$ 0$$,$$ 0$$,$$ 0.3194$$,$$ 0.125$$,$$ 0.1111$$,$$ 0$$,$$ 5.3194$$
$$x_3$$,$$ 0$$,$$ -0.2222$$,$$ 1$$,$$ 0$$,$$ 0.2222$$,$$ 0$$,$$ -0.4444$$,$$ 0$$,$$ 0.2222$$
$$s_4$$,$$ 0$$,$$ 3.6667$$,$$ 0$$,$$ 0$$,$$ 1.2083$$,$$ 0.125$$,$$ 0.3333$$,$$ 1$$,$$ 21.2083$$


Indeed, we get the same solution as if we have solved the problem afresh.

---

What if we wish to find out the range of which we can perturb $\mathbf b$?
For example, how much can we increase or decrease the amount of time we have, such that the best activities are still gaming and napping?

Suppose that we wish to find the range of $b_1$ such that $\mathbf B$ remains the optimal basis.

We need to assert that $\mathbf B^{-1} \mathbf b^* \geq 0$., where $\mathbf b^* = \mathbf b + \delta e_1$
$$
\mathbf B^{-1} \mathbf b^* =
\mathbf B^{-1} \left(\mathbf b + \delta \begin{pmatrix}1 \\ 0 \\ 0 \\ 0\end{pmatrix}\right) = 
\begin{pmatrix}
0.25 & -0.25 & 0 & 0 \\
0.375 & 0.125 & 0 & 0 \\
-0.5 & 0 & 1 & 0 \\
1.375 & 0.125 & 0 & 1
\end{pmatrix}
\left(
\begin{pmatrix}
8 \\ 4 \\ 6 \\ 3
\end{pmatrix}
+
\delta \begin{pmatrix}1 \\ 0 \\0 \\0\end{pmatrix}
\right)
=
\begin{pmatrix}1 \\ 3.5 \\2 \\14.5\end{pmatrix}
+
\delta \begin{pmatrix}0.25 \\ 0.375 \\-0.5 \\1.375\end{pmatrix}
\geq \mathbf 0
$$

(Note: $\mathbf B^{-1} \mathbf b$ can be read of the tableau, it is simply the last column).

Hence, we get the following 4 inequalities:
$$
\begin{alignedat}{1}
&1 + 0.25 \delta \geq 0 &\quad \Rightarrow  \delta \geq -4 \\ 
&3.5 + 0.375 \delta \geq 0 &\quad \Rightarrow \delta \geq -\frac{28}{3} \\
&2 - 0.5 \delta \geq 0 &\quad \Rightarrow \delta \leq 4 \\
&14.5 + 1.375 \delta \geq 0&\quad \Rightarrow \delta \geq -\frac{58}{55} \\
\end{alignedat}
$$

Hence, we get $-\frac{58}{55} \leq \delta \leq 4$.
This explains our previous situation, where the solution changed because the increase in time amount was $5 > 4$.

## Change in $\mathbf c$

Change the cost function will not affect feasibility of our solution, but may affect optimality.

Since our solution is still feasible, it will still be a BFS, and thus we can simply compute the reduced cost, and see if we need to perform more iterations.

### Example

Suppose that our new cost function is $x_1 + x_2 + x_3 + x_4$, that is we are equally happy with any of the activities.

In [6]:
costs = np.array([-1, -1, -1, -1, 0, 0, 0, 0, 0])
c_bar = reduced_cost(costs, orig_tableau)
print("New initial tableau")
print_table(c_bar, orig_tableau, variables, c=costs, c_bold=[0, 1, 2, 3])

print("Final tableau")
print_table(*simplex(c_bar, orig_tableau), variables)

New initial tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\mathbf c$$,$$\bf -1$$,$$\bf -1$$,$$\bf -1$$,$$\bf -1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$
$$\bar {\mathbf c}$$,$$\bf -1$$,$$\bf -1$$,$$\bf -1$$,$$\bf -1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$
$$s_1$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ 1$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 8$$
$$s_2$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ -3$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 4$$
$$s_3$$,$$ 1$$,$$ 1$$,$$ -2$$,$$ 0.5$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 6$$
$$s_4$$,$$ -3$$,$$ 2$$,$$ 0$$,$$ -1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 3$$


Final tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 3$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 1.75$$,$$ 0.25$$,$$ 0$$,$$ 0$$,$$ 15$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 1$$
$$x_3$$,$$ 4$$,$$ 2$$,$$ 1$$,$$ 0$$,$$ 1.5$$,$$ 0.5$$,$$ 0$$,$$ 0$$,$$ 14$$
$$s_3$$,$$ 9$$,$$ 5$$,$$ 0$$,$$ 0$$,$$ 2.875$$,$$ 1.125$$,$$ 1$$,$$ 0$$,$$ 33.5$$
$$s_4$$,$$ -3$$,$$ 2$$,$$ 0$$,$$ 0$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 1$$,$$ 4$$


The above is the optimal result after perturbing the costs.

Then the reduced cost after updating our tableau is 

In [7]:
c_bar = reduced_cost(costs, sol_tableau)
print("Updated tableau")
print_table(c_bar, sol_tableau, variables, c=costs, c_bold=[i for i in range(9)])

Updated tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\mathbf c$$,$$\bf -1$$,$$\bf -1$$,$$\bf -1$$,$$\bf -1$$,$$\bf 0$$,$$\bf 0$$,$$\bf 0$$,$$\bf 0$$,$$\bf 0$$
$$\bar {\mathbf c}$$,$$\bf 0$$,$$\bf -0.5$$,$$\bf -0.75$$,$$\bf 0$$,$$\bf 0.625$$,$$\bf -0.125$$,$$\bf 0$$,$$\bf 0$$,$$\bf 4.5$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 1$$
$$x_1$$,$$ 1$$,$$ 0.5$$,$$ 0.25$$,$$ 0$$,$$ 0.375$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$ 3.5$$
$$s_3$$,$$ 0$$,$$ 0.5$$,$$ -2.25$$,$$ 0$$,$$ -0.5$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 2$$
$$s_4$$,$$ 0$$,$$ 3.5$$,$$ 0.75$$,$$ 0$$,$$ 1.375$$,$$ 0.125$$,$$ 0$$,$$ 1$$,$$ 14.5$$


The negative reduced cost means that our solution is no longer optimal, and we can simply continue the simplex iterations from there.

In [8]:
print_table(*simplex(c_bar, sol_tableau), variables)

$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 3$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 1.75$$,$$ 0.25$$,$$ 0$$,$$ 0$$,$$ 15$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 1$$
$$x_3$$,$$ 4$$,$$ 2$$,$$ 1$$,$$ 0$$,$$ 1.5$$,$$ 0.5$$,$$ 0$$,$$ 0$$,$$ 14$$
$$s_3$$,$$ 9$$,$$ 5$$,$$ 0$$,$$ 0$$,$$ 2.875$$,$$ 1.125$$,$$ 1$$,$$ 0$$,$$ 33.5$$
$$s_4$$,$$ -3$$,$$ 2$$,$$ 0$$,$$ 0$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 1$$,$$ 4$$


Once again, we reached the same solution.

---

Suppose that we wish to find the range of value for $c_2$ such that our basis remains optimal.

Recall that we require $\bar {\mathbf c} = \mathbf c ^T - \mathbf c^T_B \mathbf B^{-1} \mathbf A \geq \mathbf 0$ for optimality.

Suppose that we are changing a non-basic variable ($c_2$ is non-basic), then notice that the 2nd term does not depend on $c_2$ as it is non-basic.

Hence, if $c_2$ is perturbed to be $c_2 + \delta$, then the resultant reduced cost is simply $\bar{\mathbf c}_2 + \delta$.

Hence, the range for which we keep our basis is $\delta \geq -\bar {\mathbf c}_i$.

Indeed, since $\bar {\mathbf c}_2 = 0$, if we increase $c_2$, our solution remains the same.

In [9]:
costs = np.array([-2, 10, -0.5, -0.5, 0, 0, 0, 0, 0])
c_bar = reduced_cost(costs, orig_tableau)
print("New initial tableau")
print_table(costs, orig_tableau, variables, c_bold=[1])

print("Final tableau")
print_table(*simplex(c_bar, orig_tableau), variables)

New initial tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ -2$$,$$\bf 10$$,$$ -0.5$$,$$ -0.5$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$
$$s_1$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ 1$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 8$$
$$s_2$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ -3$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 4$$
$$s_3$$,$$ 1$$,$$ 1$$,$$ -2$$,$$ 0.5$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 6$$
$$s_4$$,$$ -3$$,$$ 2$$,$$ 0$$,$$ -1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 3$$


Final tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 0$$,$$ 11$$,$$ 0$$,$$ 0$$,$$ 0.875$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$ 7.5$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 1$$
$$x_1$$,$$ 1$$,$$ 0.5$$,$$ 0.25$$,$$ 0$$,$$ 0.375$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$ 3.5$$
$$s_3$$,$$ 0$$,$$ 0.5$$,$$ -2.25$$,$$ 0$$,$$ -0.5$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 2$$
$$s_4$$,$$ 0$$,$$ 3.5$$,$$ 0.75$$,$$ 0$$,$$ 1.375$$,$$ 0.125$$,$$ 0$$,$$ 1$$,$$ 14.5$$


And any reduction in $c_2$ causes the optimal solution to change

In [10]:
costs = np.array([-2, -1.1, -0.5, -0.5, 0, 0, 0, 0, 0])
c_bar = reduced_cost(costs, orig_tableau)
print("New initial tableau")
print_table(costs, orig_tableau, variables, c_bold=[1])

print("Final tableau")
print_table(*simplex(c_bar, orig_tableau), variables)

New initial tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ -2$$,$$\bf -1.1$$,$$ -0.5$$,$$ -0.5$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$
$$s_1$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ 1$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 8$$
$$s_2$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ -3$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 4$$
$$s_3$$,$$ 1$$,$$ 1$$,$$ -2$$,$$ 0.5$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 6$$
$$s_4$$,$$ -3$$,$$ 2$$,$$ 0$$,$$ -1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 3$$


Final tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0.908$$,$$ 0.1284$$,$$ 0.0091$$,$$ 0.0273$$,$$ 7.9136$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 1$$
$$x_1$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0.1364$$,$$ 0.1061$$,$$ 0.0606$$,$$ -0.1515$$,$$ 1.4242$$
$$x_2$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0.3295$$,$$ 0.0341$$,$$ 0.0909$$,$$ 0.2727$$,$$ 4.1364$$
$$x_3$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0.2955$$,$$ 0.0076$$,$$ -0.4242$$,$$ 0.0606$$,$$ 0.0303$$


Suppose that we are changing a basic variable ($c_1$ for instance).
Now, both terms depend on $c_1$, hence we need to compute the reduced costs.
However, notice that since $x_1$ and $x_4$ are basic variables, their reduced costs will be $0$, *ie* $\bar {\mathbf c}_1, \bar {\mathbf c}_4 = 0$, therefore we do not need to compute those.

Changes in the cost of basic variables will affect the reduced costs of non-basic variables, thus we need to check those.
Suppose that $c_1$ becomes $c_1 + \delta$, then

$$
\begin{align}
\bar c_2 &= c_2 - \mathbf c^T _B \mathbf B^{-1} \mathbf A_2 \\
&= -1 -
\begin{pmatrix}
-0.5 & -2 + \delta & 0 & 0
\end{pmatrix}
\begin{pmatrix}
0.25 & -0.25 & 0 & 0 \\
0.375 & 0.125 & 0 & 0 \\
-0.5 & 0 & 1 & 0 \\
1.375 & 0.125 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
1 \\ 1 \\ 1 \\2
\end{pmatrix}
\\&=
-1 - 
\begin{pmatrix}
-0.5 & -2 + \delta 
\end{pmatrix}
\begin{pmatrix}
0.25 & -0.25  \\
0.375 & 0.125  \\
\end{pmatrix}
\begin{pmatrix}
1 \\ 1 
\end{pmatrix}
\\&=
-1 - 
\begin{pmatrix}
-0.5 & -2 + \delta 
\end{pmatrix}
\begin{pmatrix}
0 \\
0.5
\end{pmatrix}
\\&=
-1 - 0.5(-2 + \delta)
\\&=
-\delta \geq 0
\end{align}
$$

Thus, we get $\delta \leq 0$.
But remember that we also need to compute it for the other variables too.

$$
\bar c_3 = c_3 - \mathbf c^T _B \mathbf B^{-1} \mathbf A_3 \\
= -0.5 -
\begin{pmatrix}
-0.5 & -2
\end{pmatrix}
\begin{pmatrix}
0.25 & -0.25\\
0.375 & 0.125\\
\end{pmatrix}
\begin{pmatrix}
0.5 \\ 0.5
\end{pmatrix}
- 
\begin{pmatrix}
0 & \delta
\end{pmatrix}
\begin{pmatrix}
0.25 & -0.25\\
0.375 & 0.125\\
\end{pmatrix}
\begin{pmatrix}
0.5 \\ 0.5
\end{pmatrix}
\
\\=
-0.5 + 0.5
-0.25\delta \geq 0
$$

Hence, $\delta \leq 0$.
And we continue on to obtain the range for a change in $c_1$.

$$
\bar c_5 = c_5 - \mathbf c^T _B \mathbf B^{-1} \mathbf A_5 \\
= 0 -
\begin{pmatrix}
-0.5 & -2
\end{pmatrix}
\begin{pmatrix}
0.25 & -0.25\\
0.375 & 0.125\\
\end{pmatrix}
\begin{pmatrix}
1 \\ 0
\end{pmatrix}
- 
\begin{pmatrix}
0 & \delta
\end{pmatrix}
\begin{pmatrix}
0.25 & -0.25\\
0.375 & 0.125\\
\end{pmatrix}
\begin{pmatrix}
1 \\ 0
\end{pmatrix}
\
\\=
0.875
-0.375\delta \geq 0
$$

Hence, we get $\delta \leq \frac{7}{3}$.

$$
\bar c_6 = c_6 - \mathbf c^T _B \mathbf B^{-1} \mathbf A_6 \\
= 0 -
\begin{pmatrix}
-0.5 & -2
\end{pmatrix}
\begin{pmatrix}
0.25 & -0.25\\
0.375 & 0.125\\
\end{pmatrix}
\begin{pmatrix}
0 \\ 1
\end{pmatrix}
- 
\begin{pmatrix}
0 & \delta
\end{pmatrix}
\begin{pmatrix}
0.25 & -0.25\\
0.375 & 0.125\\
\end{pmatrix}
\begin{pmatrix}
0 \\ 1
\end{pmatrix}
\
\\=
0.125
-0.125\delta \geq 0
$$

Lastly, we get $\delta \leq 1$. 
Therefore, the range is $\delta \leq 0$.

Indeed, if we increase $c_1$, we obtain a different solution.

In [11]:
costs = np.array([-1.9, -1, -0.5, -0.5, 0, 0, 0, 0, 0])
c_bar = reduced_cost(costs, orig_tableau)
print("New initial tableau")
print_table(c_bar, orig_tableau, variables, c=costs, c_bold=[0])

print("Final tableau")
print_table(*simplex(c_bar, orig_tableau), variables)

New initial tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\mathbf c$$,$$\bf -1.9$$,$$ -1$$,$$ -0.5$$,$$ -0.5$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$
$$\bar {\mathbf c}$$,$$\bf -1.9$$,$$ -1$$,$$ -0.5$$,$$ -0.5$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$
$$s_1$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ 1$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 8$$
$$s_2$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ -3$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 4$$
$$s_3$$,$$ 1$$,$$ 1$$,$$ -2$$,$$ 0.5$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 6$$
$$s_4$$,$$ -3$$,$$ 2$$,$$ 0$$,$$ -1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 3$$


Final tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 0.1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0.875$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$ 7.5$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 1$$
$$s_3$$,$$ 9$$,$$ 5$$,$$ 0$$,$$ 0$$,$$ 2.875$$,$$ 1.125$$,$$ 1$$,$$ 0$$,$$ 33.5$$
$$s_4$$,$$ -3$$,$$ 2$$,$$ 0$$,$$ 0$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 1$$,$$ 4$$
$$x_3$$,$$ 4$$,$$ 2$$,$$ 1$$,$$ 0$$,$$ 1.5$$,$$ 0.5$$,$$ 0$$,$$ 0$$,$$ 14$$


But reducing $c_1$ retains the solution.

In [12]:
costs = np.array([-30, -1, -0.5, -0.5, 0, 0, 0, 0, 0])
c_bar = reduced_cost(costs, orig_tableau)
print("New initial tableau")
print_table(costs, orig_tableau, variables, c_bold=[0])

print("Final tableau")
print_table(*simplex(c_bar, orig_tableau), variables)

New initial tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$\bf -30$$,$$ -1$$,$$ -0.5$$,$$ -0.5$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$
$$s_1$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ 1$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 8$$
$$s_2$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ -3$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 4$$
$$s_3$$,$$ 1$$,$$ 1$$,$$ -2$$,$$ 0.5$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 6$$
$$s_4$$,$$ -3$$,$$ 2$$,$$ 0$$,$$ -1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 3$$


Final tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 0$$,$$ 14$$,$$ 7$$,$$ 0$$,$$ 11.375$$,$$ 3.625$$,$$ 0$$,$$ 0$$,$$ 105.5$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 1$$
$$x_1$$,$$ 1$$,$$ 0.5$$,$$ 0.25$$,$$ 0$$,$$ 0.375$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$ 3.5$$
$$s_3$$,$$ 0$$,$$ 0.5$$,$$ -2.25$$,$$ 0$$,$$ -0.5$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 2$$
$$s_4$$,$$ 0$$,$$ 3.5$$,$$ 0.75$$,$$ 0$$,$$ 1.375$$,$$ 0.125$$,$$ 0$$,$$ 1$$,$$ 14.5$$


### A quicker way

In fact, there is a quicker way to obtain the range.
If we look at the tableau:

In [13]:
print_table(
    sol_c_bar,
    sol_tableau,
    variables,
    c_bold=[i for i in range(9)],
    tableau_bold=(1, 0, 1, 8),
)

$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$\bf 0$$,$$\bf 0$$,$$\bf 0$$,$$\bf 0$$,$$\bf 0.875$$,$$\bf 0.125$$,$$\bf 0$$,$$\bf 0$$,$$\bf 7.5$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 1$$
$$x_1$$,$$\bf 1$$,$$\bf 0.5$$,$$\bf 0.25$$,$$\bf 0$$,$$\bf 0.375$$,$$\bf 0.125$$,$$\bf 0$$,$$\bf 0$$,$$ 3.5$$
$$s_3$$,$$ 0$$,$$ 0.5$$,$$ -2.25$$,$$ 0$$,$$ -0.5$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 2$$
$$s_4$$,$$ 0$$,$$ 3.5$$,$$ 0.75$$,$$ 0$$,$$ 1.375$$,$$ 0.125$$,$$ 0$$,$$ 1$$,$$ 14.5$$


Suppose that our range is in the form of $a \geq b \delta$.

Notice that the $\bar {\mathbf c}$ coincides with the $a$, while the values in the 2nd row (the row associated with the variable $x_1$) coincides with $b$.
Hence, we can actually obtain the range simply by reading off the tableau.

So for example, the range of $c_4$ is simply
$$
0.875 \geq 0.25 \delta \Rightarrow \delta \leq 3.5 \\
0.125 \geq -0.25 \delta \Rightarrow \delta \geq -0.5 \\
$$

## Change in non-basic column of $\mathbf A$

Suppose than an entry $A_{i,j}$ was modified to $A_{i,j} + \delta$.
We assume that feasibility is maintained.
In that case, optimality still may be affected.

And only the reduced cost in the modified column would be affected.

Computing the difference, we would see that the new reduced cost is $\bar c_j - \delta p_i$.
Where $i,j$ is the row, column index of the modified cell.

If feasibility is not maintained, then we perform dual simplex iteration to obtain the new solution.

### Example

In [14]:
new_tableau = orig_tableau.copy()
new_tableau[1, 1] = 3
c_bar = reduced_cost(orig_costs, new_tableau)
print("New initial tableua")
print_table(c_bar, new_tableau, variables, tableau_bold=(1, 1, 1, 1))

print("Final tableau")
print_table(*simplex(c_bar, new_tableau), variables)

New initial tableua


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ -2$$,$$ -1$$,$$ -0.5$$,$$ -0.5$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$
$$s_1$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ 1$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 8$$
$$s_2$$,$$ 2$$,$$\bf 3$$,$$ 0.5$$,$$ -3$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 4$$
$$s_3$$,$$ 1$$,$$ 1$$,$$ -2$$,$$ 0.5$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 6$$
$$s_4$$,$$ -3$$,$$ 2$$,$$ 0$$,$$ -1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 3$$


Final tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 0$$,$$ 0.25$$,$$ 0$$,$$ 0$$,$$ 0.875$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$ 7.5$$
$$x_4$$,$$ 0$$,$$ -0.5$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 1$$
$$x_1$$,$$ 1$$,$$ 0.75$$,$$ 0.25$$,$$ 0$$,$$ 0.375$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$ 3.5$$
$$s_3$$,$$ 0$$,$$ 0.5$$,$$ -2.25$$,$$ 0$$,$$ -0.5$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 2$$
$$s_4$$,$$ 0$$,$$ 3.75$$,$$ 0.75$$,$$ 0$$,$$ 1.375$$,$$ 0.125$$,$$ 0$$,$$ 1$$,$$ 14.5$$


Firstly, we compute the new $\bar {\mathbf c}_2$
$$
\bar {\mathbf c'}_2 = \mathbf c^T - \mathbf c^T_B \mathbf B^{-1} \left(\mathbf  A_2 + \begin{pmatrix} 0 \\ 2 \\ 0 \\ 0 \end{pmatrix} \right) 
= \bar {\mathbf c}_2 - \mathbf c^T_B \mathbf B^{-1} \begin{pmatrix} 0 \\ 2 \\ 0 \\ 0 \end{pmatrix}
=
0 - 
\begin{pmatrix}
-2 & -1 & 0 & 0
\end{pmatrix}
\begin{pmatrix}
0.25 & -0.25 & 0 & 0 \\
0.375 & 0.125 & 0 & 0 \\
-0.5 & 0 & 1 & 0 \\
1.375 & 0.125 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
0 \\ 2 \\ 0\\ 0
\end{pmatrix}
= 0.25
$$

Then we compute the new column in the solved tableau.


$$
\mathbf B^{-1} \left(\mathbf  A_2 + \begin{pmatrix} 0 \\ 2 \\ 0 \\ 0 \end{pmatrix} \right) 
=
\begin{pmatrix}
0 \\ 0.5 \\ 0.5 \\ 3.5
\end{pmatrix}
+
\begin{pmatrix}
0.25 & -0.25 & 0 & 0 \\
0.375 & 0.125 & 0 & 0 \\
-0.5 & 0 & 1 & 0 \\
1.375 & 0.125 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
0 \\ 2 \\ 0\\ 0
\end{pmatrix}
=
\begin{pmatrix}
0 \\ 0.5 \\ 0.5 \\ 3.5
\end{pmatrix}
+
\begin{pmatrix}
-0.5 \\ 0.25 \\ 0\\ 0.25
\end{pmatrix}
=
\begin{pmatrix}
-0.5 \\ 0.75 \\ 0.5\\ 3.75
\end{pmatrix}
$$

Since our reduced costs was positive, simply replacing all the entries above in the solution tableau would give us our new solution.

---

Suppose now instead, we wish to find the range of $\delta$ such that changes to the $A_{2, 2}$ entry won't change the basis.
Once again, we compute the below to ensure optimality.

$$
\bar {\mathbf c'}_2 = \mathbf c^T - \mathbf c^T_B \mathbf B^{-1} \left(\mathbf  A_2 + \begin{pmatrix} 0 \\ \delta \\ 0 \\ 0 \end{pmatrix} \right) 
=
0 - 
\begin{pmatrix}
-2 & -1 & 0 & 0
\end{pmatrix}
\begin{pmatrix}
0.25 & -0.25 & 0 & 0 \\
0.375 & 0.125 & 0 & 0 \\
-0.5 & 0 & 1 & 0 \\
1.375 & 0.125 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
0 \\ \delta \\ 0\\ 0
\end{pmatrix}
= 0.25 \delta \geq 0
$$

Hence, we require $\delta \geq 0$.

Indeed, just a small decrease in that entry causes the solution to change.

In [15]:
new_tableau = orig_tableau.copy()
new_tableau[1, 1] = 0.9
c_bar = reduced_cost(orig_costs, new_tableau)
print("New initial tableua")
print_table(c_bar, new_tableau, variables, tableau_bold=(1, 1, 1, 1))

print("Final tableau")
print_table(*simplex(c_bar, new_tableau), variables)

New initial tableua


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ -2$$,$$ -1$$,$$ -0.5$$,$$ -0.5$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$
$$s_1$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ 1$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 8$$
$$s_2$$,$$ 2$$,$$\bf 0.9$$,$$ 0.5$$,$$ -3$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 4$$
$$s_3$$,$$ 1$$,$$ 1$$,$$ -2$$,$$ 0.5$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 6$$
$$s_4$$,$$ -3$$,$$ 2$$,$$ 0$$,$$ -1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 3$$


Final tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0.8791$$,$$ 0.1254$$,$$ 0.0011$$,$$ 0.0034$$,$$ 7.5519$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.2417$$,$$ -0.2509$$,$$ -0.0023$$,$$ -0.0068$$,$$ 0.8962$$
$$x_1$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0.1399$$,$$ 0.1064$$,$$ 0.0616$$,$$ -0.1486$$,$$ 1.4683$$
$$x_2$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0.3307$$,$$ 0.0342$$,$$ 0.0912$$,$$ 0.2737$$,$$ 4.1505$$
$$x_3$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0.2957$$,$$ 0.0076$$,$$ -0.4242$$,$$ 0.0608$$,$$ 0.0334$$


## Adding new variable

This time, feasibility is maintained, but optimality may be affected.
Thus, it is straightforward for us to perform another simplex iteration.

### Example

Suppose that I've discovered a new activity: exercising.
I obtain a happiness of $3$, but at the cost of 1 unit of time, hunger and energy.

In [16]:
costs = np.concatenate((orig_costs[:-1], [-3], orig_costs[-1:]))
tableau = np.hstack(
    (orig_tableau[:, :-1], np.array([1, 1, 1, 0]).reshape((4, 1)), orig_tableau[:, -1:])
)
c_bar = reduced_cost(costs, tableau)
new_variables = variables + ["x_5"]
print("New initial tableau")
print_table(costs, tableau, new_variables, c_bold=[8], tableau_bold=(0, 8, 4, 1))

print("Final tableau")
print_table(*simplex(c_bar, tableau), new_variables)

New initial tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$x_5$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ -2$$,$$ -1$$,$$ -0.5$$,$$ -0.5$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$\bf -3$$,$$ 0$$
$$s_1$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ 1$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$\bf 1$$,$$ 8$$
$$s_2$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ -3$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0$$,$$\bf 1$$,$$ 4$$
$$s_3$$,$$ 1$$,$$ 1$$,$$ -2$$,$$ 0.5$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0$$,$$\bf 1$$,$$ 6$$
$$s_4$$,$$ -3$$,$$ 2$$,$$ 0$$,$$ -1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$\bf 0$$,$$ 3$$


Final tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$x_5$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 3.6$$,$$ 2$$,$$ 0$$,$$ 0$$,$$ 2.025$$,$$ 0.575$$,$$ 0.4$$,$$ 0$$,$$ 0$$,$$ 20.9$$
$$x_3$$,$$ 0.4$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0.35$$,$$ 0.05$$,$$ -0.4$$,$$ 0$$,$$ 0$$,$$ 0.6$$
$$x_5$$,$$ 1.8$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0.575$$,$$ 0.225$$,$$ 0.2$$,$$ 0$$,$$ 1$$,$$ 6.7$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$
$$s_4$$,$$ -3$$,$$ 2$$,$$ 0$$,$$ 0$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 4$$


We simply augment the same data to our solved tableau, and continue the simplex iterations if need be.

The column that we need to add to our tableau is 

$$
\mathbf B^{-1} A_9 = 
\begin{pmatrix}
0.25 & -0.25 & 0 & 0 \\
0.375 & 0.125 & 0 & 0 \\
-0.5 & 0 & 1 & 0 \\
1.375 & 0.125 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
1 \\ 1 \\ 1 \\0
\end{pmatrix}
=
\begin{pmatrix}
0 \\ 0.5 \\ 0.5 \\ 1.5
\end{pmatrix}
$$

In [17]:
tableau = np.hstack(
    (
        sol_tableau[:, :-1],
        np.array([0, 0.5, 0.5, 1.5]).reshape((4, 1)),
        sol_tableau[:, -1:],
    )
)
c_bar = reduced_cost(costs, tableau)
print("New initial tableau")
print_table(c_bar, tableau, new_variables, c_bold=[8], tableau_bold=(0, 8, 4, 1))

print("Final tableau")
print_table(*simplex(c_bar, tableau), new_variables)

New initial tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$x_5$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0.875$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$\bf -2$$,$$ 7.5$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$\bf 0$$,$$ 1$$
$$x_1$$,$$ 1$$,$$ 0.5$$,$$ 0.25$$,$$ 0$$,$$ 0.375$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$\bf 0.5$$,$$ 3.5$$
$$s_3$$,$$ 0$$,$$ 0.5$$,$$ -2.25$$,$$ 0$$,$$ -0.5$$,$$ 0$$,$$ 1$$,$$ 0$$,$$\bf 0.5$$,$$ 2$$
$$s_4$$,$$ 0$$,$$ 3.5$$,$$ 0.75$$,$$ 0$$,$$ 1.375$$,$$ 0.125$$,$$ 0$$,$$ 1$$,$$\bf 1.5$$,$$ 14.5$$


Final tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$x_5$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 3.6$$,$$ 2$$,$$ 0$$,$$ 0$$,$$ 2.025$$,$$ 0.575$$,$$ 0.4$$,$$ 0$$,$$ 0$$,$$ 20.9$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$
$$x_3$$,$$ 0.4$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0.35$$,$$ 0.05$$,$$ -0.4$$,$$ 0$$,$$ 0$$,$$ 0.6$$
$$x_5$$,$$ 1.8$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0.575$$,$$ 0.225$$,$$ 0.2$$,$$ 0$$,$$ 1$$,$$ 6.7$$
$$s_4$$,$$ -3$$,$$ 2$$,$$ 0$$,$$ 0$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 4$$


## Adding new constraint

Adding new constraint won't affect optimality, but can affect feasibility.

Suppose that our solution satisfy the new constraint, then it will still remain an optimal solution.

However, if our solution does not satisfy this new constraint, then we perform the following

1. Introduce a new variable $x_{n+1}$
    * This is equivalent to a slack variable
2. Set the coefficient of this variable to be $0$ in all the constraints, except in the new constraint who has a coefficient of $1$
3. Make $\mathbf x_B \cup \{x_{n+1}\}$ the new basis
    * *ie* zero out all the columns associated with basic variables in the new constraint
4. If this leads to infeasibility, then perform dual simplex iterations

### Example

Suppose that now I have another resource that I have to account for: money.

And we have the following new LP:

In [18]:
# We need to introduce slack variable
costs = np.concatenate((orig_costs[:-1], [0], orig_costs[-1:]))
tableau = np.hstack((orig_tableau[:, :-1], np.zeros((4, 1)), orig_tableau[:, -1:]))
tableau = np.vstack((tableau, [1, 0, 2, 0, 0, 0, 0, 0, 1, 3]))
c_bar = reduced_cost(costs, tableau)
new_variables = variables + ["s_5"]

print("New initial tableau")
print_table(c_bar, tableau, new_variables, c=costs, tableau_bold=(4, 0, 1, 10))

print("Final tableau")
print_table(*simplex(c_bar, tableau), new_variables)

New initial tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$s_5$$,$$Solution$$
$$\mathbf c$$,$$ -2$$,$$ -1$$,$$ -0.5$$,$$ -0.5$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$
$$\bar {\mathbf c}$$,$$ -2$$,$$ -1$$,$$ -0.5$$,$$ -0.5$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$
$$s_1$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ 1$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 8$$
$$s_2$$,$$ 2$$,$$ 1$$,$$ 0.5$$,$$ -3$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 4$$
$$s_3$$,$$ 1$$,$$ 1$$,$$ -2$$,$$ 0.5$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 6$$
$$s_4$$,$$ -3$$,$$ 2$$,$$ 0$$,$$ -1$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 3$$
$$s_5$$,$$\bf 1$$,$$\bf 0$$,$$\bf 2$$,$$\bf 0$$,$$\bf 0$$,$$\bf 0$$,$$\bf 0$$,$$\bf 0$$,$$\bf 1$$,$$\bf 3$$


Final tableau


$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$s_5$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0.875$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 7.5$$
$$x_2$$,$$ 0$$,$$ 1$$,$$ -3.5$$,$$ 0$$,$$ 0.75$$,$$ 0.25$$,$$ 0$$,$$ 0$$,$$ -2$$,$$ 1$$
$$x_1$$,$$ 1$$,$$ 0$$,$$ 2$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 3$$
$$s_3$$,$$ 0$$,$$ 0$$,$$ -0.5$$,$$ 0$$,$$ -0.875$$,$$ -0.125$$,$$ 1$$,$$ 0$$,$$ 1$$,$$ 1.5$$
$$s_4$$,$$ 0$$,$$ 0$$,$$ 13$$,$$ 0$$,$$ -1.25$$,$$ -0.75$$,$$ 0$$,$$ 1$$,$$ 7$$,$$ 11$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$


From our previous solution, we add the new constraint

In [19]:
tableau = np.hstack((sol_tableau[:, :-1], np.zeros((4, 1)), sol_tableau[:, -1:]))
tableau = np.vstack((tableau, [1, 0, 2, 0, 0, 0, 0, 0, 1, 3]))
print_table(costs, tableau, new_variables, tableau_bold=(4, 0, 1, 10))

$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$s_5$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ -2$$,$$ -1$$,$$ -0.5$$,$$ -0.5$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$
$$x_4$$,$$ 1$$,$$ 0.5$$,$$ 0.25$$,$$ 0$$,$$ 0.375$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 3.5$$
$$s_3$$,$$ 0$$,$$ 0.5$$,$$ -2.25$$,$$ 0$$,$$ -0.5$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 2$$
$$s_4$$,$$ 0$$,$$ 3.5$$,$$ 0.75$$,$$ 0$$,$$ 1.375$$,$$ 0.125$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 14.5$$
$$s_5$$,$$\bf 1$$,$$\bf 0$$,$$\bf 2$$,$$\bf 0$$,$$\bf 0$$,$$\bf 0$$,$$\bf 0$$,$$\bf 0$$,$$\bf 1$$,$$\bf 3$$


Zero out all the basis columns in the new constraint.

In [20]:
tableau[-1] -= tableau[1]
print_table(costs, tableau, new_variables)

$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$s_5$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ -2$$,$$ -1$$,$$ -0.5$$,$$ -0.5$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$
$$x_1$$,$$ 1$$,$$ 0.5$$,$$ 0.25$$,$$ 0$$,$$ 0.375$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 3.5$$
$$s_3$$,$$ 0$$,$$ 0.5$$,$$ -2.25$$,$$ 0$$,$$ -0.5$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 0$$,$$ 2$$
$$s_4$$,$$ 0$$,$$ 3.5$$,$$ 0.75$$,$$ 0$$,$$ 1.375$$,$$ 0.125$$,$$ 0$$,$$ 1$$,$$ 0$$,$$ 14.5$$
$$s_5$$,$$ 0$$,$$ -0.5$$,$$ 1.75$$,$$ 0$$,$$ -0.375$$,$$ -0.125$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ -0.5$$


Now that it has becomes infeasible, we simply perform dual simplex iterations on it.

In [21]:
c_bar = reduced_cost(costs, tableau)

print_table(*dual_simplex(c_bar, tableau), new_variables)

$$$$,$$x_1$$,$$x_2$$,$$x_3$$,$$x_4$$,$$s_1$$,$$s_2$$,$$s_3$$,$$s_4$$,$$s_5$$,$$Solution$$
$$\bar {\mathbf c}$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0.875$$,$$ 0.125$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 7.5$$
$$x_4$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 0.25$$,$$ -0.25$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$
$$x_1$$,$$ 1$$,$$ 0$$,$$ 2$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 0$$,$$ 1$$,$$ 3$$
$$s_3$$,$$ 0$$,$$ 0$$,$$ -0.5$$,$$ 0$$,$$ -0.875$$,$$ -0.125$$,$$ 1$$,$$ 0$$,$$ 1$$,$$ 1.5$$
$$s_4$$,$$ 0$$,$$ 0$$,$$ 13$$,$$ 0$$,$$ -1.25$$,$$ -0.75$$,$$ 0$$,$$ 1$$,$$ 7$$,$$ 11$$
$$x_2$$,$$ 0$$,$$ 1$$,$$ -3.5$$,$$ 0$$,$$ 0.75$$,$$ 0.25$$,$$ 0$$,$$ 0$$,$$ -2$$,$$ 1$$


Once again, we arrived at the same solution.